# TraceWin parsing

**IN DEVELOPMENT**

Comments after `;`

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from tracewin import parsers

In [3]:
# These are the elements that are assigned for parsing. 
# Note that there are more elements in tracewin.elements
parsers.ele_parser

{'quad': tracewin.elements.Quad,
 'aperture': tracewin.elements.Aperture,
 'chopper': tracewin.elements.Chopper,
 'drift': tracewin.elements.Drift,
 'field_map': tracewin.elements.Field_Map,
 'field_map_path': tracewin.elements.Field_Map_Path,
 'freq': tracewin.elements.Freq,
 'lattice': tracewin.elements.Lattice,
 'lattice_end': tracewin.elements.Lattice_End,
 'set_sync_phase': tracewin.elements.Set_Sync_Phase,
 'thin_steering': tracewin.elements.Thin_Steering}

In [4]:
# Read some lattice lines
LINES = open('data/linac.dat').readlines()

In [5]:
eles = []
for line in LINES:
    ele = parsers.parse_lattice_line(line)
    if ele.ele_type == 'end':
        print('end found')
        break
    eles.append(ele)
    
# Complete set
set(ele.ele_type for ele in eles)

end found


{'aperture',
 'chopper',
 'drift',
 'field_map',
 'field_map_path',
 'freq',
 'lattice',
 'lattice_end',
 'marker',
 'quad',
 'set_sync_phase',
 'thin_steering'}

In [6]:
# Parsed elements
eles[0:14]

[Field_Map_Path(path='Fields'),
 DummyElement(ele_type='marker', line=''),
 Drift(L=250.0, R=16.0, Ry=0.0, Rx_shift=0.0, Ry_shift=0.0),
 Drift(L=1e-20, R=14.0, Ry=0.0, Rx_shift=0.0, Ry_shift=0.0),
 DummyElement(ele_type='marker', line=''),
 DummyElement(ele_type='marker', line=''),
 Freq(f=162.5),
 DummyElement(ele_type='marker', line=''),
 DummyElement(ele_type='marker', line=''),
 DummyElement(ele_type='marker', line=''),
 Drift(L=50.0, R=15.0, Ry=0.0, Rx_shift=0.0, Ry_shift=0.0),
 Aperture(dx=15.0, dy=15.0, n=1),
 Quad(L=100.0, G=12.2, R=15.0, theta=0.0, g3=0.0, g4=0.0, g5=0.0, g6=0.0, GRF=0.0),
 Aperture(dx=15.0, dy=15.0, n=1)]

# as TraceWin

Call the `.as_tracewin` method on each element to reproduce the original input.

In [7]:
for ele in eles[0:20]:
    if ele.ele_type == 'marker':
        continue
    #print(ele, ele.ele_type)
    print(ele.as_tracewin)

FIELD_MAP_PATH Fields
DRIFT 250.0 16.0 0.0 0.0 0.0
DRIFT 1e-20 14.0 0.0 0.0 0.0
FREQ 162.5
DRIFT 50.0 15.0 0.0 0.0 0.0
APERTURE 15.0 15.0 1
QUAD 100.0 12.2 15.0 0.0 0.0 0.0 0.0 0.0 0.0
APERTURE 15.0 15.0 1
DRIFT 35.0 15.0 0.0 0.0 0.0
DRIFT 1e-08 14.0 0.0 0.0 0.0
DRIFT 35.0 15.0 0.0 0.0 0.0
APERTURE 15.0 15.0 1
QUAD 100.0 -10.88 15.0 0.0 0.0 0.0 0.0 0.0 0.0


# as Bmad

In [8]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [9]:
def bmad_lines(eles):
    
    idrift = 0
    iquad = 0
    ifieldmap = 0
    isteer = 0
    
    lines = []
    names = []
    freq = 0
    
    for ele in eles:
        t = ele.ele_type
        
        if t == 'drift':
            idrift += 1
            name = f'D{idrift}'
            line = f'{name}: drift, L = {ele.L/1000}'
            
        elif t == 'quad':
            iquad += 1
            name = f'Q{iquad}'
            line = f'{name}: quad, L = {ele.L/1000}'
            
        elif t == 'field_map':
            ifieldmap += 1
            name = f'FM{ifieldmap}'
            line = f'{name}: lcavity, L = {ele.L/1000}, rf_frequency = {freq*1e6}'
            
        elif t =='thin_steering':
            isteer += 1
            name = f'K{isteer}'
            line = f'{name}: kicker, L = 0'
        elif t == 'freq':
            freq = ele.f
            continue
        else:
            name = 'BAD'
            line = ''
            continue
            
        
         
        # Collect
        lines.append(line)
        names.append(name)
    
    latlines = []
    for c in chunks(names, 8):
        line = ', '.join(c)
        latlines.append(line)
    latline = 'lat: line = (\n'
    latline += ',\n'.join(latlines)
    latline += ')'

    lines.append(latline)
    
    return lines
            
bmad_lines(eles);

In [10]:
with open('test.bmad', 'w') as f:
    for line in bmad_lines(eles):
        f.write(line + '\n')

In [11]:
!head test.bmad

D1: drift, L = 0.25
D2: drift, L = 1e-23
D3: drift, L = 0.05
Q1: quad, L = 0.1
D4: drift, L = 0.035
D5: drift, L = 1.0000000000000001e-11
D6: drift, L = 0.035
Q2: quad, L = 0.1
D7: drift, L = 0.05
D8: drift, L = 0.009


# Fieldmaps

TODO

In [12]:
fm = []
g  = []
for ele in eles:
    if ele.ele_type == 'field_map':
        fm.append(ele.filename)
        g.append(ele.geom)

In [13]:
set(fm)

{'HWR-SOL-ANLMAP',
 'HWRDonut',
 'QWR-2012-02',
 'SSR13D_#72',
 'SSR2_beta047_v26',
 'SSR_SOL_cut',
 'beta063D_#43',
 'beta092'}

In [14]:
set(g)

{10, 70, 7700}

In [15]:
gkeys = ('static electric field',
'static magnetic field',
'RF electric field',
'RF magnetic field',
'3D aperture map')
def parse_geom(geom):
    return dict(zip(gkeys, list(reversed(f'{geom:05}'))))

parse_geom(7700)    

{'static electric field': '0',
 'static magnetic field': '0',
 'RF electric field': '7',
 'RF magnetic field': '7',
 '3D aperture map': '0'}

In [16]:
parse_geom(10)

{'static electric field': '0',
 'static magnetic field': '1',
 'RF electric field': '0',
 'RF magnetic field': '0',
 '3D aperture map': '0'}

In [17]:
parse_geom(70)

{'static electric field': '0',
 'static magnetic field': '7',
 'RF electric field': '0',
 'RF magnetic field': '0',
 '3D aperture map': '0'}

In [18]:
# Cleanup
!rm test.bmad